# Исследование данных об инвестиции венчурных фондов в компании-стартапы

## Данные

Самостоятельная работа основана на базе данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы, и основана на датасете Startup Investments https://www.kaggle.com/justinas/startup-investments.

## Цель

Проанализировать данные о фондах и инвестициях, произвести выгрузки данных и ответить на поставленные вопросы с помощью SQL.

## Задания


### Задание 1
Посчитайте, сколько компаний закрылось.


In [ ]:
'''
SELECT COUNT(name)
FROM company
WHERE status ='closed';
'''

### Задание 2
Отобразите количество привлечённых средств для новостных компаний США.
Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.



In [ ]:
'''
SELECT SUM(funding_total) AS S
FROM company
WHERE country_code = 'USA'
AND category_code = 'news'
GROUP BY name 
ORDER BY S DESC;
'''

### Задание 3
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.



In [ ]:
'''
SELECT SUM(price_amount)
FROM acquisition
WHERE EXTRACT(YEAR FROM acquired_at::DATE) IN (2011,2012,2013)
AND term_code = 'cash';  
'''

### Задание 4
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.



In [ ]:
'''
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
'''

### Задание 5
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'



In [ ]:
'''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';
'''

### Задание 6
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.



In [ ]:
'''
SELECT SUM(funding_total) AS S,
       country_code 
FROM company
GROUP BY country_code
ORDER BY S DESC;
'''

### Задание 7
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.


In [ ]:
'''
SELECT funded_at AS date,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY date
HAVING MIN(raised_amount) > 0
AND MIN(raised_amount) != MAX(raised_amount);
'''

### Задание 8
Создайте поле с категориями:

Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.

Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.

Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.


In [ ]:
'''
SELECT *,
       CASE 
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies< 20 THEN 'low_activity'
       END    
FROM fund ;
'''

### Задание 9
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. 

Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
'''
SELECT ROUND(AVG(investment_rounds)),
       activity
FROM           
(SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund) AS Q
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
'''

### Задание 10
Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.

Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему.

Для фильтрации диапазона по годам используйте оператор BETWEEN.


In [ ]:
'''
SELECT *
FROM
(SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies) AS AVG
FROM fund
WHERE EXTRACT(YEAR FROM founded_at::DATE) IN (2010,2011,2012)
GROUP BY country_code 
HAVING MIN(invested_companies) <> 0) AS Q
ORDER BY AVG DESC, country_code
LIMIT 10;
'''

### Задание 11
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.



In [ ]:
'''
SELECT p.first_name,
       p.last_name,
       e.instituition 
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;
'''

### Задание 12
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. 

Выведите название компании и число уникальных названий учебных заведений. 

Составьте топ-5 компаний по количеству университетов.



In [ ]:
'''
WITH
p AS (SELECT id, 
             company_id
     FROM people),
e AS (SELECT person_id,
             instituition
     FROM education)


SELECT c.name,
       COUNT(DISTINCT e.instituition) AS instituition
FROM company AS c 
LEFT JOIN p ON c.id = p.company_id 
LEFT JOIN e ON p.id = e.person_id
GROUP BY c.name
ORDER BY count(DISTINCT e.instituition) DESC
LIMIT 5;;
'''

### Задание 13
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.



In [ ]:
'''
SELECT DISTINCT c.name
FROM 
 (SELECT id, name 
  FROM company  
  WHERE status = 'closed') AS c
INNER JOIN
 (SELECT company_id       
  FROM funding_round
  WHERE is_first_round = 1
  AND is_first_round = is_last_round) AS fr ON c.id = fr.company_id;
'''



### Задание 14
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.



In [ ]:
'''
SELECT DISTINCT p.id
FROM people AS p
left join 
 (SELECT id, name 
  FROM company  
  WHERE status = 'closed') AS c ON p.company_id = c.id
INNER JOIN
 (SELECT company_id       
  FROM funding_round
  WHERE is_first_round = 1
  AND is_first_round = is_last_round) AS fr ON c.id = fr.company_id;
'''

### Задание 15
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.



In [ ]:
'''
WITH
fr AS (SELECT company_id       
        FROM funding_round
        WHERE is_first_round = 1
        AND is_first_round = is_last_round),
c AS  (SELECT id, name 
       FROM company  
       WHERE status = 'closed'),
p AS (SELECT *
       FROM people)

SELECT instituition, p.id 
FROM education AS e
JOIN
p ON e.person_id=p.id
LEFT JOIN 
  c ON p.company_id = c.id
INNER JOIN
  fr ON c.id = fr.company_id
GROUP by instituition, p.id 
ORDER BY p.id;
'''

### Задание 16
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания.



In [ ]:
'''
SELECT p.id,
COUNT(e.instituition)
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN
(SELECT c.id
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE STATUS ='closed'
AND is_first_round = 1
AND is_last_round = 1
GROUP BY c.id)
GROUP BY p.id
HAVING COUNT(DISTINCT e.instituition) >0;
'''

### Задание 17
Дополните предыдущий запрос и выведите среднее число учебных заведений, которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.



In [ ]:
'''
SELECT AVG (COUNT)
FROM
(SELECT p.id,
        COUNT(e.instituition)
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN
(SELECT c.id
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE STATUS ='closed'
AND is_first_round = 1
AND is_last_round = 1
GROUP BY c.id)
GROUP BY p.id
HAVING COUNT(DISTINCT e.instituition) >0) as w;
'''

### Задание 18
Напишите похожий запрос: выведите среднее число учебных заведений, которые окончили сотрудники компании Facebook.



In [ ]:
'''
WITH base AS 
    (SELECT p.id,
    COUNT(e.instituition)
    FROM people AS p
    RIGHT JOIN education AS e ON p.id = e.person_id
    WHERE p.company_id IN 
                        (SELECT id
                        FROM company
                        WHERE name = 'Facebook')
                        GROUP BY p.id)

SELECT AVG(COUNT)
FROM base;
'''



### Задание 19
Составьте таблицу из полей:

- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.



In [ ]:
'''
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN company AS c ON c.id = i.company_id
LEFT JOIN fund AS f ON i.fund_id = f.id
INNER JOIN 
(SELECT*
FROM funding_round
WHERE funded_at BETWEEN '2012-01-01' AND '2013-12-31')
AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6;
'''

### Задание 20
Выгрузите таблицу, в которой будут такие поля:

- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в алфавитном порядке. Ограничьте таблицу первыми десятью записями.



In [ ]:
'''
WITH acquiring AS
    (SELECT c.name AS buyer,
    a.price_amount AS price,
    a.id AS KEY
    FROM acquisition AS a
    LEFT JOIN company AS c ON a.acquiring_company_id = c.id
    WHERE a.price_amount > 0),
acquired AS
    (SELECT c.name AS acquisition,
    c.funding_total AS investment,
    a.id AS KEY
    FROM acquisition AS a
    LEFT JOIN company AS c ON a.acquired_company_id = c.id
    WHERE c.funding_total > 0)
    
SELECT acqn.buyer,
       acqn.price,
       acqd.acquisition,
       acqd.investment,
       ROUND(acqn.price / acqd.investment) AS uplift
FROM acquiring AS acqn
JOIN acquired AS acqd ON acqn.KEY = acqd.KEY
ORDER BY price DESC, acquisition
LIMIT 10;
'''

### Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год.

Выведите также номер месяца, в котором проходил раунд финансирования.



In [ ]:
'''
SELECT  c.name AS social_co,
EXTRACT (MONTH FROM fr.funded_at) AS funding_month
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
AND fr.funded_at BETWEEN '2010-01-01' AND '2013-12-31'
AND fr.raised_amount <> 0;
'''

### Задание 22
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды.

Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.


In [ ]:
'''
WITH
fundings AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE)) AS funding_month,
                    COUNT(DISTINCT f.id) AS us_funds
             FROM fund AS f
             LEFT JOIN investment AS i ON f.id = i.fund_id
             LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id
             WHERE f.country_code = 'USA'
             AND EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
             GROUP BY funding_month),
acquisitions AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS funding_month,
                       COUNT(acquired_company_id) AS bought_co,
                       SUM(price_amount) AS sum_total
                 FROM acquisition
                 WHERE EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013
                 GROUP BY funding_month)
SELECT fnd.funding_month,
       fnd.us_funds,
       acq.bought_co,
       acq.sum_total
FROM fundings AS fnd
LEFT JOIN acquisitions AS acq ON fnd.funding_month = acq.funding_month;

'''

### Задание 23
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. 

Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.


In [ ]:

'''
WITH
y_2011 AS (SELECT country_code,
                 AVG(funding_total) AS y_2011                 
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at::DATE) = 2011
          GROUP BY country_code),
y_2012 AS (SELECT country_code,
                 AVG(funding_total) AS y_2012                 
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at::DATE) = 2012
          GROUP BY country_code),  
y_2013 AS (SELECT country_code,
                 AVG(funding_total) AS y_2013                 
          FROM company
          WHERE EXTRACT(YEAR FROM founded_at::DATE) = 2013
          GROUP BY country_code) 

SELECT y_2011.country_code AS country,
       y_2011.y_2011,
       y_2012.y_2012,
       y_2013.y_2013
FROM y_2011 
JOIN y_2012 ON y_2011.country_code = y_2012.country_code
JOIN y_2013 ON y_2012.country_code = y_2013.country_code
ORDER BY y_2011.y_2011 DESC
'''